# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [6]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../data/fires/forestfires.csv', header = None, names = columns))


# Get X and Y

Create the features data frame and target data.

In [ ]:
X = fires_dt.drop('area', axis=1)  
y = fires_dt['area']   
            
print("Features (X):")
print(X.head())
print("\nTarget (y):")
print(y.head())

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [7]:
# preproc1
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


num_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
cat_features = ['month', 'day']


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])


categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preproc1 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)


print(preproc1)


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [8]:
# preproc2

from sklearn.preprocessing import FunctionTransformer

log_transformer = FunctionTransformer(np.log1p, validate=True)

numeric_transformer_2 = Pipeline(steps=[
    ('log', log_transformer),
    ('scaler', StandardScaler())
])


preproc2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_2, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)


print(preproc2)


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [ ]:
# Pipeline A = preproc1 + baseline

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor


baseline_regressor = KNeighborsRegressor()


pipeline_a = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])


print(pipeline_a)

In [ ]:
# Pipeline B = preproc2 + baseline


pipeline_b = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])


print(pipeline_b)


In [10]:
# Pipeline C = preproc1 + advanced model

from sklearn.ensemble import RandomForestRegressor


advanced_regressor = RandomForestRegressor(random_state=42)


pipeline_c = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])


print(pipeline_c)


In [9]:
# Pipeline D = preproc2 + advanced model


pipeline_d = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])

print(pipeline_d)


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for Pipeline A
param_grid_a = {
    'regressor__n_neighbors': [3, 5, 7, 9]
}

# Perform grid search with cross-validation for Pipeline A
grid_search_a = GridSearchCV(pipeline_a, param_grid_a, cv=5, n_jobs=-1, verbose=2)
grid_search_a.fit(X_train, y_train)

# Display best parameters and score for Pipeline A
print("Pipeline A - Best Parameters:", grid_search_a.best_params_)
print("Pipeline A - Best Score:", grid_search_a.best_score_)


In [ ]:
# Define parameter grid for Pipeline B
param_grid_b = {
    'regressor__n_neighbors': [3, 5, 7, 9]
}

# Perform grid search with cross-validation for Pipeline B
grid_search_b = GridSearchCV(pipeline_b, param_grid_b, cv=5, n_jobs=-1, verbose=2)
grid_search_b.fit(X_train, y_train)

# Display best parameters and score for Pipeline B
print("Pipeline B - Best Parameters:", grid_search_b.best_params_)
print("Pipeline B - Best Score:", grid_search_b.best_score_)


In [ ]:
# Define parameter grid for Pipeline C
param_grid_c = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30]
}

# Perform grid search with cross-validation for Pipeline C
grid_search_c = GridSearchCV(pipeline_c, param_grid_c, cv=5, n_jobs=-1, verbose=2)
grid_search_c.fit(X_train, y_train)

# Display best parameters and score for Pipeline C
print("Pipeline C - Best Parameters:", grid_search_c.best_params_)
print("Pipeline C - Best Score:", grid_search_c.best_score_)


In [ ]:
# Define parameter grid for Pipeline D
param_grid_d = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30]
}

# Perform grid search with cross-validation for Pipeline D
grid_search_d = GridSearchCV(pipeline_d, param_grid_d, cv=5, n_jobs=-1, verbose=2)
grid_search_d.fit(X_train, y_train)

# Display best parameters and score for Pipeline D
print("Pipeline D - Best Parameters:", grid_search_d.best_params_)
print("Pipeline D - Best Score:", grid_search_d.best_score_)


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
1. Evaluate
Determine the Best Performing Model:
To identify the model with the best performance, compare the results from your hyperparameter tuning process. Look at the cross-validation scores obtained for each model. The model with the highest cross-validation score is considered the best performing. This score reflects how well the model generalizes to unseen data.
2. Export
Save the Best Performing Model to a Pickle File:
Once you have identified the model with the best performance, you need to save this model for future use. This is done by serializing the model into a file using a format like pickle. This file can then be loaded later to make predictions or to continue working with the model without having to retrain it.
3. Explain
Use SHAP Values to Explain the Best Performing Model:

Select an Observation from the Test Set and Explain Its Prediction:
Choose a specific data point from your test set and use SHAP values to explain the prediction made by the model for this data point. SHAP values help in understanding which features of this particular observation had the most influence on the model's prediction. This process involves identifying and visualizing the impact of each feature on the prediction for that observation.

Determine the Most and Least Important Features Across the Training Set:
To understand the importance of features across the entire training set, you analyze the SHAP values for all the features used in the model. This analysis reveals which features generally have the most and least impact on the model’s predictions across the dataset. It helps in identifying which features are consistently influential and which ones contribute less to the model’s performance.

Remove Features and Test Model Performance:
Consider removing certain features from the model and then evaluate the performance of the modified model. This involves training a new model without the removed features and comparing its performance against the original model. This comparison helps determine whether the removed features were contributing positively to the model’s performance or if their exclusion has negligible impact.

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.